<a href="https://colab.research.google.com/github/adarsh415/Tensorflow2.x/blob/master/RNN_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import optimizers, layers, datasets, metrics, models
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

TensorFlow 2.x selected.


In [2]:
(x_train, y_train), (x_test, y_test) = datasets.imdb.load_data()

17465344/17464789 [==============================] - 2s 0us/step


In [0]:
config = {'sent_len':100,
'vocab':100000,
'embed_size':100,
'cell_size':[32,64,128],
'num_layers':3
}

In [0]:
X_train_pad = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=config['sent_len'])
X_test_pad = keras.preprocessing.sequence.pad_sequences(x_test)

In [0]:
class Sentiment(models.Model):
  def __init__(self, kwargs):
    super(Sentiment, self).__init__()
    self.cell_size=kwargs['cell_size']
    self.num_layers=kwargs['num_layers']
    self.embed_size=kwargs['embed_size']
    self.sent_len=kwargs['sent_len']
    self.vocab=kwargs['vocab']

    assert len(self.cell_size) == self.num_layers

    self.cells = [layers.LSTMCell(self.cell_size[i]) for i in range(self.num_layers)]
    self.rnn = layers.RNN(self.cells, return_state=True)
    self.embedding = layers.Embedding(self.vocab,self.embed_size, input_length=self.sent_len, mask_zero=True)
    self.dense = keras.layers.Dense(1)

  def call(self, x):
    x = self.embedding(x)
    x=self.rnn(x)
    x = self.dense(x[0])
    return  x

In [0]:
rnn = Sentiment(config)

In [0]:
rnn.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'] )

In [8]:
rnn.fit(x=X_train_pad,y=y_train, batch_size=64, epochs=20, validation_data=(X_test_pad,y_test), verbose=1)

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 597s 24ms/sample - loss: 0.4134 - accuracy: 0.7956 - val_loss: 0.3284 - val_accuracy: 0.8658
Epoch 2/20
25000/25000 [==============================] - 587s 23ms/sample - loss: 0.1872 - accuracy: 0.9267 - val_loss: 0.3241 - val_accuracy: 0.8667
Epoch 3/20
25000/25000 [==============================] - 588s 24ms/sample - loss: 0.0855 - accuracy: 0.9690 - val_loss: 0.4863 - val_accuracy: 0.8550
Epoch 4/20
25000/25000 [==============================] - 583s 23ms/sample - loss: 0.0410 - accuracy: 0.9864 - val_loss: 0.6033 - val_accuracy: 0.8426
Epoch 5/20
25000/25000 [==============================] - 588s 24ms/sample - loss: 0.0327 - accuracy: 0.9885 - val_loss: 0.5798 - val_accuracy: 0.8441
Epoch 6/20
25000/25000 [==============================] - 602s 24ms/sample - loss: 0.0205 - accuracy: 0.9930 - val_loss: 0.6817 - val_accuracy: 0.8400
Epoch 7/20
25000/25000 [====================

In [9]:
prediction = rnn.predict(X_test_pad,128, verbose=1)

25000/25000 [==============================] - 213s 9ms/sample


In [21]:
score = rnn.evaluate(x=X_test_pad,y=y_test, batch_size=128)

25000/25000 [==============================] - 218s 9ms/sample - loss: 1.1227 - accuracy: 0.8337


In [23]:
print(f'test loss: {score[0]} and accuracy: {score[1]}')

test loss: 1.122720851135254 and accuracy: 0.8337200284004211
